In [26]:
import pandas as pd
import numpy as np
import spacy
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
with open('Conversa do WhatsApp com [CPF] ENGLISH.txt') as f:
    df = f.readlines()

In [28]:
pattern1 = r'\d{2}/\d{2}/\d{4}\s\d{2}:\d{2}'
pattern2 = r'(\(|\+)*\d{1,3}\)* \(*\d{2,5}\)* \d{2,7}((\-|\s)\d{2,5})*((\-|\s)\d{2,5})*|[a-zA-Z]+'
pattern3 = r': (\w|\s|\W){2,}'
patternnumber = r'\s\+\d*\s(\(|\d)*(\s|\))(\d|\s)*(\s|\-)\d*'

In [29]:
date = re.compile(pattern1)

In [30]:
datelist = []
userlist = []
messagelist = []
banlist = []
leftlist = []
invlist= []
linklist = []
j=0
for i in range(len(df)):
    if 'entrou usando o link de convite' in df[i]:
        linklist.append(df[i])
    try:
        date.match(df[i]).start() == 0
        searchuser = re.search(pattern2,df[i]).group()
        searchmessage = re.search(pattern3, df[i]).group()[2:].lower().rstrip('\n')
        userlist.append(searchuser)
        messagelist.append(searchmessage)
        datelist.append(df[i][0:16])
    
    except AttributeError:
        if 'removeu' in df[i]:
            banlist.append(df[i])
        if 'saiu' in df[i]:
            leftlist.append(df[i])
        if 'adicionou' in df[i]:
            invlist.append(df[i])
        pass
 

In [31]:
zlist = list(zip(datelist,userlist,messagelist))

In [32]:
dataframewhats = pd.DataFrame(zlist, columns = ['Time' , 'User', 'Messages']) 

In [33]:
dataframewhats['Time'] = dataframewhats['Time'].apply(lambda x: datetime.strptime(x,"%d/%m/%Y %H:%M"))

In [34]:
dataframewhats['Messages'] = dataframewhats['Messages'].apply(lambda x: np.nan if len(str(x))==0 else x)

In [35]:
dataframewhats.isnull().values.any()

False

## File Table

In [36]:
datefile = []
userfile = []
for i in range(len(dataframewhats)):
    if 'oculto' in dataframewhats['Messages'][i]:
        datefile.append(dataframewhats['Time'][i])
        userfile.append(dataframewhats['User'][i])

In [37]:
zlist2 = list(zip(datefile,userfile))

In [38]:
dataframefile =  pd.DataFrame(zlist2, columns = ['Time' , 'User']) 

## Ban Table

In [39]:
cban = []
dateban = []
for mes in banlist:
    x = re.search(r'(\w{3,}\s)*\w{3,} removeu', mes).group()
    y = x.replace(' removeu','')
    cban.append(y)
    dateban.append(mes[0:16])

In [40]:
uban = []

for mes in banlist:
    x = re.search(r'removeu (\(|\+)*\d{1,3}\)* \(*\d{2,5}\)* \d{2,7}((\-|\s)\d{2,5})*((\-|\s)\d{2,5})*', mes) or re.search(r'removeu [a-zA-Z]{3,}', mes)
    y = x.group()
    z = y.replace('removeu ','')
    uban.append(z)

In [41]:
zlist1 = list(zip(dateban,cban,uban))

In [42]:
dataframewhatsban = pd.DataFrame(zlist1, columns = ['Time','Coordinator', 'User']) 

In [43]:
dataframewhatsban['Time'] = dataframewhatsban['Time'].apply(lambda x: datetime.strptime(x,"%d/%m/%Y %H:%M"))

In [44]:
dataframewhatsban['Coordinator'] = dataframewhatsban['Coordinator'].apply(lambda x: 'Ravik' if x == 'Você' else x)

In [45]:
dataframewhats.isnull().values.any()

False

## Left Table

In [46]:
dateleft = []
userleft = []
for mes in leftlist:
    dateleft.append(mes[0:16])
    x = re.search('(\(|\+)*\d{1,3}\)* \(*\d{2,5}\)* \d{2,7}((\-|\s)\d{2,5})*((\-|\s)\d{2,5})*|[a-zA-Z]+', mes)
    y = x.group()
    userleft.append(y)

In [47]:
zlist3 = list(zip(dateleft,userleft))

In [48]:
dataframeleft = pd.DataFrame(zlist3, columns = ['Time' , 'User']) 

In [49]:
dataframeleft['Time'] = dataframeleft['Time'].apply(lambda x: datetime.strptime(x,"%d/%m/%Y %H:%M"))

## Tagging Table

In [50]:
usertagged = []
datetagged = []
for i in range(len(dataframewhats)):
    if '@' in dataframewhats['Messages'][i]:
        try:
            x = re.search(r'@\d{3,}', dataframewhats['Messages'][i]).group()
            usertagged.append(x)
            datetagged.append(dataframewhats['Time'][i])
        except:
            pass

In [51]:
zlist4 = list(zip(datetagged,usertagged))

In [52]:
dataframetagged = pd.DataFrame(zlist4, columns = ['Time','User']) 

In [53]:
# it's not necessary making any conversion to timedate format

## Adding new colunms

In [58]:
dataframewhats['Day_of_Week'] = dataframewhats['Time'].apply(lambda x: x.strftime('%a'))

In [59]:
dataframewhats['Date'] = dataframewhats['Time'].apply(lambda x: x.strftime('%b %Y'))

In [60]:
dataframewhats['Year'] = dataframewhats['Time'].apply(lambda x: x.year)

## People invited Table

In [69]:
cinv = []
dateinv = []
useradd = []
for mes in invlist:
    if re.search(patternnumber,mes):    #use re.search() instead of re.match()
        for i in re.finditer(patternnumber, mes):
            x = i.group(0)
            useradd.append(x.replace('\n','').replace(' +','+'))
            y = re.search(pattern2, mes).group()
            z = y.replace(' adicionou','')
            cinv.append(z)
            dateinv.append(mes[0:16])
    else:
        w = re.search(r'adicionou\s[a-zA-Z]{1,}', mes)
        useradd.append(w.group(0).replace('adicionou ',''))
        dateinv.append(mes[0:16])
        k = w.group(0).replace('adicionou ','')
        cinv.append(k)

In [70]:
zlist5 = list(zip(dateinv,cinv,useradd))

In [71]:
dataframewhatsinvite = pd.DataFrame(zlist5, columns = ['Time','Coordinator', 'User']) 

In [72]:
dataframewhatsinvite['Coordinator'] = dataframewhatsinvite['Coordinator'].apply(lambda x: 'Ravik' if x == 'Voc' else x)

## People joining by the link

In [73]:
userlink = []
datelink = []
for mes in linklist:
    datelink.append(mes[0:16])
    x = re.search(r'\u200e\+\d*\s*\(*\d*\)*\s*\d*-*\d*\s*\d*\s*\d*\s*\d*\-*\d*',mes) or re.search('\u200e[a-zA-Z]{1,}',mes)
    y = x.group(0).replace('\u200e','').rstrip()
    userlink.append(y)

In [74]:
zlist6 = list(zip(datelink,userlink))

In [75]:
dataframewhatslink = pd.DataFrame(zlist6, columns = ['Time' , 'User']) 

In [96]:
dataframewhatslink['User'] = dataframewhatslink['User'].apply(lambda x: 'Joao Paulo' if x == 'Joao' else x)

## Region Table

In [76]:
import phonenumbers
from phonenumbers import geocoder
from phonenumbers.phonenumberutil import region_code_for_country_code
import pycountry

In [77]:
parse_country = []
for user in dataframewhats['User']:
    try:
        x = phonenumbers.parse(user)
        parse_country.append(x)

    except:
        parse_country.append(np.nan)

In [78]:
dataframewhats['parse_country'] = parse_country

In [79]:
code_country_parse = []
for c in dataframewhats['parse_country']:
    try:
        x = region_code_for_country_code(c.country_code)
        code_country_parse.append(x)
    except:
        code_country_parse.append(np.nan)

In [80]:
dataframewhats['code_country_parse'] = code_country_parse

In [81]:
dataframewhats['code_country_parse'].fillna('BR',inplace=True)

In [82]:
country_name = []
for c in dataframewhats['code_country_parse']:
    x = pycountry.countries.get(alpha_2=c).name
    country_name.append(x)
    

In [83]:
dataframewhats['country_name'] = country_name

In [84]:
codebrazil = []
for r in dataframewhats['User']:
    if '+55' in r:
        x = re.search(r'\+55\s\d{2}',r).group().replace('+55 ','')
        codebrazil.append(x)
    elif re.search(r'[a-zA-Z]',r):
        codebrazil.append('85')
    else:
        codebrazil.append(np.nan)

In [85]:
dataframewhats['codebrazil'] = codebrazil

## Sentiment Analysis

In [86]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ravik/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [87]:
dataframewhats['scores'] = dataframewhats['Messages'].apply(lambda x: sid.polarity_scores(x))

In [88]:
dataframewhats['Compound'] = dataframewhats['scores'].apply(lambda x: x['compound'])

In [984]:
dataframewhats['comp_score'] = dataframewhats['Compound'].apply(lambda x: 'pos' if x >=0 else 'neg')

## Export Data

In [62]:
dataframefile.to_csv('dataframefiletotableau.csv')

In [61]:
dataframewhats.to_csv('WhatsappFinaltotableauComplete.csv')

In [63]:
dataframetagged.to_csv('dataframetaggedtotableau.csv')

In [64]:
dataframewhatsban.to_csv('dataframebantotableau.csv')

In [66]:
dataframeleft.to_csv('dataframelefttotableau.csv')

In [89]:
dataframewhatslink.to_csv('dataframelinktotableau.csv')

In [90]:
dataframewhatsinvite.to_csv('dataframeinvitetotableau.csv')